<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course2/week2/Optimization_methods_v1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최적화 기법 #

지금까지 여러분은 파라미터를 업데이트하고 비용(cost)을 최소화하는 방법으로 고전적인 경사 하강법(Gradient Descent)만을 사용했습니다. 이번 과제에서는, 학습 속도를 빠르게 만들어주고, 더 작은 비용 함수의 최종 결과를 도출할 수 있는 보다 발전된 최적화 함수를 사용해보도록 하겠습니다. 좋은 최적화 알고리즘을 쓴다는 것은 좋은 결과를 얻을 때 까지 하루의 시간을 쓰느냐, 혹은 단지 몇 시간만만에 학습을 끝내느냐의 큰 차이를 만들어냅니다.

경사 하강법은, 비용 함수 $J$에서 내리막을 이동하는 것과 같습니다. 아래 그림과 비슷하다고 생각해봅시다.

<img src="arts/cost.jpg" style="width:650px;height:300px;">

**그림 1** </u>: **cost를 최소화 한다는 것은, 언덕이 많은 울퉁불퉁한 땅에서 가장 낮은 지점을 찾는것과 비슷합니다.**
<br> 
훈련의 각 단계에서 여러분은 가능한 가장 낮은 지점에 도달하기 위해, 특정 방향에 따라 파라미터를 업데이트합니다.

**표기법** : 늘 그랬듯이, 어떤 변수 `a` 에 대해 미분계수 `da`를 다음과 같이 표현합니다. $\frac{\partial J}{\partial a } = $ `da`

시작하기에 앞서, 필요한 라이브러리를 다운받아보겠습니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets

from opt_utils_v1a import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from opt_utils_v1a import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset
from testCases import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## 1. 일반적인 경사 하강법 ##

기계학습에서 가장 간단한 최적화 기법 중 하나가 경사 하강법입니다. 만약 모든 샘플 데이터 $m$에 대하여 일괄적으로 경사 하강법을 수행한다면, 이를 배치 경사 하강법(Batch Gradient Descent) 라고 부릅니다.

**연습 문제**: 지금까지 배웠던 일반적인 경사 하강법을 구현해봅시다. 공식은 아래와 같습
니다.
$l = 1, ..., L$ 인 $l$에 대하여: 
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$

$L$은 모델의 층(레이어)의 갯수이고, $\alpha$는 학습률입니다. 모든 파라미터는 `parameters` 딕셔너리에 저장됩니다. iterator $l$은 0에서 시작하는 반면, 첫 번째 파라미터는 $W^{[1]}$과 $b^{[1]}$ 에서 시작한다는 것을 잊지 마세요. `for` 문을 코딩 할때 `l`을 `l+1`로 바꿔야 합니다. 


In [ ]:
# GRADED FUNCTION: update_parameters_with_gd

def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- the learning rate, scalar.
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """

    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = None
        parameters["b" + str(l+1)] = None
        ### END CODE HERE ###
        
    return parameters

In [ ]:
parameters, grads, learning_rate = update_parameters_with_gd_test_case()

parameters = update_parameters_with_gd(parameters, grads, learning_rate)
print("W1 =\n" + str(parameters["W1"]))
print("b1 =\n" + str(parameters["b1"]))
print("W2 =\n" + str(parameters["W2"]))
print("b2 =\n" + str(parameters["b2"]))

**모범 답안**:

```
W1 =
[[ 1.63535156 -0.62320365 -0.53718766]
 [-1.07799357  0.85639907 -2.29470142]]
b1 =
[[ 1.74604067]
 [-0.75184921]]
W2 =
[[ 0.32171798 -0.25467393  1.46902454]
 [-2.05617317 -0.31554548 -0.3756023 ]
 [ 1.1404819  -1.09976462 -0.1612551 ]]
b2 =
[[-0.88020257]
 [ 0.02561572]
 [ 0.57539477]]
```

이 경사 하강법에 약간의 변형을 준 방법이 SGD(Stocahstic Gradient Descent: 확률적 경사 하강법)로, 각 미니 배치에 단 하나의 샘플만 있는 미니 배치 경사 하강법과 동일합니다. 이 방법은 위에서 구현한 파라미터 업데이트 규칙이 도중에 변경되지 않습니다. 일반적인 경사 하강법과 다른 점은, 전체 학습 세트가 아닌 하나의 학습 예제에 대해서만 기울기를 계산한다는 것입니다. 아래 코드는 SGD와 일반적인 (Batch) 경사 하강법의 차이점을 보여줍니다.

- **(Batch) 경사 하강법**:

``` python
X = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    # Forward propagation
    a, caches = forward_propagation(X, parameters)
    # Compute cost.
    cost += compute_cost(a, Y)
    # Backward propagation.
    grads = backward_propagation(a, caches, parameters)
    # Update parameters.
    parameters = update_parameters(parameters, grads)
        
```

- **SGD(Stochastic Gradient Descent)**:

```python
X = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    for j in range(0, m):
        # Forward propagation
        a, caches = forward_propagation(X[:,j], parameters)
        # Compute cost
        cost += compute_cost(a, Y[:,j])
        # Backward propagation
        grads = backward_propagation(a, caches, parameters)
        # Update parameters.
        parameters = update_parameters(parameters, grads)
```


확률적 경사 하강법(Stochastic Gradient Descent)에서는, 기울기를 업데이트하기 전 단 한개의 훈련 데이터만 사용합니다. 만약 여러분의 훈련 세트가 매우 크다면, SGD가 일반적인 경사 하강법보다 더 빠를 수 있습니다. 하지만 파라미터는 최소값을 향해 매끄럽게 수렴하지 않고 진동합니다. 아래 그림은 이에 대한 설명입니다.

<img src="images/kiank_sgd.png" style="width:750px;height:250px;">

**그림 1** : <u> SGD vs GD</u><br>
"+"는 비용함수의 최소값을 의미합니다. SGD는 최소값에 수렴하는 과정에서 많은 진동이 발생하지만, 각각 단계를 계산하는 속도는 GD가 SGD보다 확실히 빠릅니다. GD가 모든 훈련 세트(batch)를 사용하는 반면, SGD는 한개의 훈련 데이터만 활용하기 때문입니다.

<br>

**메모**: SGD는 총 3개의 반복문을 활용해서도 구현할 수 있습니다.
1. 훈련의 반복 횟수 만큼의 for 문
2. $m$ 개의 훈련 데이터 갯수 만큼의 for 문
3. 모델의 레이어의 갯수만큼 파라미터를 업데이트($(W^{[1]},b^{[1]})$ ~ $(W^{[L]},b^{[L]})$)하는 for 문

실제로는 업데이트를 위해서 전체 학습 세트를 한번에 사용하거나, 하나의 학습 예제를 사용하는 두 방법 다 최적의 방법은 아닙니다. 둘 사이의 어떤 갯수만큼 묶어 학습하면 더 빠른 결과를 얻을 수 있습니다. 미니 배치 경사 하강법(Mini-Batch Gradient Descent)은 각 단계에 대해서 중간 갯수의 훈련 데이터 묶음을 사용합니다. 미니 배치 경사 하강법을 사용하면 개별 훈련 데이터를 반복하는 대신 미니 배치를 반복합니다.

<img src="arts/kiank_minibatch.png" style="width:750px;height:250px;">

**그림 2**: <u>**SGD vs Mini-Batch**</u>
<br>
"+"는 비용 함수의 최솟값을 의미합니다. 미니 배치를 최적화 알고리즘에 사용하면, 더 빠른 최적화로 이어집니다.

**기억해야 할 것**:
- 미니 배치(Mini-Batch) 경사 하강법과 일반적인 경사 하강법(GD), 확률적 경사 하강법(SGD) 사이의 차이점은, 업데이트 단계에서 사용하는 훈련 데이터의 갯수입니다.
- 하이퍼 파라미터 $\alpha$는 학습률을 의미하며, 튜닝해야할 필요가 있습니다.
- 잘 전환된 미니 배치 크기의 경우, 일반적으로 GD 혹은 SGD보다 성능이 뛰어납니다. 이는 훈련 세트가 큰 경우 특히 성능이 더 좋습니다.

## 2. 미니 배치 경사 하강법(Mini-Batch Gradient Descent) ##

이제 훈련 세트 (X, Y)에 대해 어떻게 미니 배치를 만드는지 알아봅시다.

두 가지 단계로 나뉘어져 있습니다.

- **셔플링(Shuffle)** : 아래 그림과 같이, 훈련 세트 (X, Y)의 섞인 버전을 만듭니다. X, Y 각 컬럼이 의미하는 바는 훈련 세트의 특성과 결과값일 것입니다. 무작위로 섞는 과정은 X와 Y 사이에서 동기적으로 일어난다는(세트로 이루어집니다) 것에 주의하세요. 셔플링 뒤 X의 i번 째 배열은 Y의 i번 째 데이터가 되어야 합니다. 셔플링 단계에서는 훈련 데이터들이 무작위로 다른 미니 배치로 분할되도록 합니다.
<img src="arts/kiank_shuffle.png" style="width:550px;height:300px;">

- **분할(Partition)** : 셔플링된 (X, Y) 데이터를 `mini_batch_size`(이번 예제에서는 64로 설정됩니다)의 크기를 갖고 있는 미니 배치로 분할하는 단계입니다. 훈련 세트의 총 개수는 `mini_batch_size`로 나누어떨어지지 않을 수 있습니다. 그렇게 되면, 마지막 미니 배치의 갯수는 좀 더 작아지겠죠. 이 부분에 대해서 걱정할 필요는 없습니다. 만약 마지막 미니 배치의 크기가 다른 `mini_batch_size`보다 작을 경우, 아래 그림을 보면 됩니다.
<img src="arts/kiank_partition.png" style="width:550px;height:300px;">


**연습 문제** : `random_mini_batches` 함수를 구현해보세요. 1단계인 셔플링 단계는 미리 구현되어 있습니다. 분할 단계에 대한 보조적인 설명을 위해서, 첫 번째와 두 번째 미니 배치를 선택하는 코드를 첨부했습니다.
```python
first_mini_batch_X = shuffled_X[:, 0 : mini_batch_size]
second_mini_batch_X = shuffled_X[:, mini_batch_size : 2 * mini_batch_size]
...
```

마지막 미니 배치가 `mini_batch_size=64`보다 작다는 점에 유의하세요. $\lfloor s \lfloor$를 $s$를 가장 작은 정수로 내림(rounded down) 한 수랴고 하겠습니다. 이는 파이썬 코드로 `math.floor(s)`로 표현할 수 있습니다. 만약 전체 훈련 세트의 개수가 `mini_batch_size=64`의 곱이 아니라면, $\lfloor \frac{m}{mini\_batch\_size}\rfloor$ 개의 완전한 미니 배치(64개의 사이즈를 가진)와, $m-mini_\_batch_\_size \times \lfloor \frac{m}{mini\_batch\_size}\rfloor$개의 사이즈를 가진 마지막 미니 배치가 만들어질 것입니다.




In [ ]:
# GRADED FUNCTION: random_mini_batches

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = None
        mini_batch_Y = None
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = None
        mini_batch_Y = None
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
X_assess, Y_assess, mini_batch_size = random_mini_batches_test_case()
mini_batches = random_mini_batches(X_assess, Y_assess, mini_batch_size)

print ("shape of the 1st mini_batch_X: " + str(mini_batches[0][0].shape))
print ("shape of the 2nd mini_batch_X: " + str(mini_batches[1][0].shape))
print ("shape of the 3rd mini_batch_X: " + str(mini_batches[2][0].shape))
print ("shape of the 1st mini_batch_Y: " + str(mini_batches[0][1].shape))
print ("shape of the 2nd mini_batch_Y: " + str(mini_batches[1][1].shape)) 
print ("shape of the 3rd mini_batch_Y: " + str(mini_batches[2][1].shape))
print ("mini batch sanity check: " + str(mini_batches[0][0][0][0:3]))

**모범 답안**:

<table style="width:50%"> 
    <tr>
    <td > <b>shape of the 1st mini_batch_X</b> </td> 
           <td > (12288, 64) </td> 
    </tr> 
    <tr>
    <td > <b>shape of the 2nd mini_batch_X</b> </td> 
           <td > (12288, 64) </td> 
    </tr> 
    <tr>
    <td > <b>shape of the 3rd mini_batch_X</b> </td> 
           <td > (12288, 20) </td> 
    </tr>
    <tr>
    <td > <b>shape of the 1st mini_batch_Y </b> </td> 
           <td > (1, 64) </td> 
    </tr> 
    <tr>
    <td > <b>shape of the 2nd mini_batch_Y </b> </td> 
           <td > (1, 64) </td> 
    </tr> 
    <tr>
    <td > <b> shape of the 3rd mini_batch_Y</b> </td> 
           <td > (1, 20) </td> 
    </tr> 
    <tr>
    <td > <b>mini batch sanity check</b> </td> 
           <td > [ 0.90085595 -0.7612069   0.2344157 ] </td> 
    </tr>
</table>

**기억해야 할 것** :
- 미니 배치를 만드는데는 셔플링(Shuffling)과 분할(Partitioning)의 총 두 단계가 필요합니다.
- 1개의 미니 배치 사이즈로, 2의 제곱수가 주로 선택됩니다 (e.g., 16, 32, 64, 128, ...).

## 3. 모멘텀 (Momentum) ##

미니 배치 경사 하강법은 전체 데이터셋의 일부 부분집합에 대해서 훈련을 진행한 이후 파라미터를 업데이트합니다. 그렇기 때문에 업데이트의 방향에 약간의 차이가 있습니다. 이는 미니 배치 경사 하강법이 반복되면서 최솟값으로 수렴되는 과정에서 진동을 야기합니다. **모멘텀(Momentum)** 을 사용하면 이 진동을 줄일 수 있습니다.

모멘텀은 진동을 줄이고 업데이트를 보다 부드러운 방향으로 이어지게 하기 위하여 이전 단계에서 계산되었던 기울기를 고려합니다. 변수 $v$에 지난 기울기의 방향을 저장합니다. 공식에서 이 값은 이전 단계에서의 기울기 지수 가중 평균이 됩니다. 또한 $v$는 물리학적으로는 언덕의 경사 / 경사 방향에 따라 속도를 높이는 공의 '속도' 로 생각할 수도 있습니다.

<img src="arts/opt_momentum.png" style="width:400px;height:250px;">

**그림 3**: 빨간 화살표는 모멘텀을 사용한 미니 배치 경사 하강법의 각 단계가 수행될 때마다의 기울기 방향입니다. 파란색 점선은 일반적인 미니 배치 경사 하강법에서 나타나는 기울기 방향입니다. 일반적인 기울기를 따르기보다는, 계산된 기울기가 $v$에 영향을 미치도록 한 다음 $v$의 방향만큼 업데이트 해줍니다.

<br>

**연습 문제**: 속도(velocity)를 초기화합니다. `v`는 0으로 초기화된 파이썬 딕셔너리 변수입니다. 이 딕셔너리의 키는, 기존 `grads` 딕셔너리의 키와 동일($l = 1, ..., L$)합니다.
```python
v["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])
v["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])
```
**주의**: for 문에을 돌릴 때 `l` 변수는 서 0부터 시작하지만, v 딕셔너리에 접근할 때에는 `v["dW1"]` 과 `v["db1]` 을 첫 번째 변수로 접근한다는 것에 유의하세요. for문에서 `l` 을 `l+1` 로 바꿔야 합니다.


In [ ]:
# GRADED FUNCTION: initialize_velocity

def initialize_velocity(parameters):
    """
    Initializes the velocity as a python dictionary with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    
    Returns:
    v -- python dictionary containing the current velocity.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    
    # Initialize velocity
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        v["dW" + str(l+1)] = None
        v["db" + str(l+1)] = None
        ### END CODE HERE ###
        
    return v

In [ ]:
parameters = initialize_velocity_test_case()

v = initialize_velocity(parameters)
print("v[\"dW1\"] =\n" + str(v["dW1"]))
print("v[\"db1\"] =\n" + str(v["db1"]))
print("v[\"dW2\"] =\n" + str(v["dW2"]))
print("v[\"db2\"] =\n" + str(v["db2"]))

**모범 답안**:

```
v["dW1"] =
[[ 0.  0.  0.]
 [ 0.  0.  0.]]
v["db1"] =
[[ 0.]
 [ 0.]]
v["dW2"] =
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
v["db2"] =
[[ 0.]
 [ 0.]
 [ 0.]]
```

**연습 문제**: 모멘텀을 적용한 값으로 파라미터를 업데이트해봅시다. 업데이트 공식은 아래와 같습니다.

$l = 1, ..., L$인 $l$에 대하여,
$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}\tag{3}$$

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}\tag{4}$$

$L$은 모델의 레이어의 개수고, $\beta$는 모멘텀 값, 그리고 $\alpha$는 학습률입니다. 모든 파라미터들은 `parameters` 딕셔너리에 저장되어야 합니다. 위와 마찬가지로, for 문에서의 반복 변수 `l`은 0부터 시작하는데 반해 가중치 및 bias 행렬은 $W^{[1]}$ 과 $b^{[1]}$ 로 1부터 시작한다는 것을 주의하세요. for문에서 `l` 을 `l+1` 로 바꿔야 합니다.



In [ ]:
# GRADED FUNCTION: update_parameters_with_momentum

def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    """
    Update parameters using Momentum
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- python dictionary containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta -- the momentum hyperparameter, scalar
    learning_rate -- the learning rate, scalar
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- python dictionary containing your updated velocities
    """

    L = len(parameters) // 2 # number of layers in the neural networks
    
    # Momentum update for each parameter
    for l in range(L):
        
        ### START CODE HERE ### (approx. 4 lines)
        # compute velocities
        v["dW" + str(l+1)] = None
        v["db" + str(l+1)] = None
        # update parameters
        parameters["W" + str(l+1)] = None
        parameters["b" + str(l+1)] = None
        ### END CODE HERE ###
        
    return parameters, v

In [ ]:
parameters, grads, v = update_parameters_with_momentum_test_case()

parameters, v = update_parameters_with_momentum(parameters, grads, v, beta = 0.9, learning_rate = 0.01)
print("W1 = \n" + str(parameters["W1"]))
print("b1 = \n" + str(parameters["b1"]))
print("W2 = \n" + str(parameters["W2"]))
print("b2 = \n" + str(parameters["b2"]))
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = v" + str(v["db2"]))

**모범 답안**:

```
W1 = 
[[ 1.62544598 -0.61290114 -0.52907334]
 [-1.07347112  0.86450677 -2.30085497]]
b1 = 
[[ 1.74493465]
 [-0.76027113]]
W2 = 
[[ 0.31930698 -0.24990073  1.4627996 ]
 [-2.05974396 -0.32173003 -0.38320915]
 [ 1.13444069 -1.0998786  -0.1713109 ]]
b2 = 
[[-0.87809283]
 [ 0.04055394]
 [ 0.58207317]]
v["dW1"] = 
[[-0.11006192  0.11447237  0.09015907]
 [ 0.05024943  0.09008559 -0.06837279]]
v["db1"] = 
[[-0.01228902]
 [-0.09357694]]
v["dW2"] = 
[[-0.02678881  0.05303555 -0.06916608]
 [-0.03967535 -0.06871727 -0.08452056]
 [-0.06712461 -0.00126646 -0.11173103]]
v["db2"] = v[[ 0.02344157]
 [ 0.16598022]
 [ 0.07420442]]
```

**다음을 기억하세요**:
- 속도가 0으로 초기화되기 때문에, 알고리즘은 실제 속도를 계산하고 다음 단계를 수행하기 위해 몇 회의 반복 이 필요하비낟.
- $\beta$가 0이면 모멘텀이 없는 일반적인 경사 하강법이 됩니다.

**$\beta$를 무슨 값으로 설정해야 하나요?**:
- 모멘텀 $\beta$가 크면 클수록, 지난 기울기를 현재 단계의 계산에 더 크게 반영하기 때문에 파라미터 업데이트가 더 부드럽게 이루어집니다. 하지만 $\beta$가 너무 크면 업데이트가 과하게 매끄러워질 수도 있습니다.
- $\beta$로 가장 일반적으로 사용되는 값은 0.8에서 0.999 사이입니다. 이 $\beta$를 별로 튜닝하고 싶은 마음이 없다면, 0.9로 초기화 하는 것이 가장 합리적인 선택일 것입니다.
- 모델에 대한 최적의 $\beta$값을 튜닝하는 것은 비용 함수 J의 측면을 줄이는 가장 효과적인 값을 확인하기 위해 여러 회 시도해야 할 수 있습니다.

**기억해야 할 것**:
- 모멘텀은 지난 계산에서의 기울기(Gradient)들을 고려하여 현재의 기울기를 계산하기 때문에 기울기가 하강하는 방향이 매끄럽습니다. 이 방법은 배치 경사 하강법, 미니 배치 경사 하강법 및 확률적 경사 하강법 등에 적용할 수 있습니다.

## 4. Adam ##

**Adam**은 인공신경망을 훈련할 때 가장 효율적인 최적화 알고리즘 중 하나입니다. 이 방법은 RMSProp과 모멘텀을 합친 방식으로 훈련을 진행합니다.

**Adam 최적화 기법의 작동방식**:
1. 먼저 지난 기울기(Gradient)들에 대한 지수 가중 평균을 계산하여, $v$ 변수(편향 보정을 하기 전)에 저장합니다. 편향 보정을 한 이후의 값은 $v^{corrected}$에 저장합니다.
2. 지난 기울기들의 **'제곱'**에 대한 지수 가중 평균을 계산하여, $s$ 변수(편향 보정 전)에 저장합니다. 편향 보정 이후의 값은 $s^{corrected}$에 저장합니다.
3. 1번에서 계산딘 값과 2번에서 계산된 값을 혼합하여 결정된 방향으로 파라미터를 업데이트합니다.

공식으로 확인하면 다음과 같습니다.

$l = 1, ..., L$ 인 $l$에 대하여,
$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$

**표기법**:
- $t$는 Adam 알고리즘을 수행할 횟수입니다.
- $L$은 모델의 레이어 갯수입니다.
- $\beta_1$ 과 $\beta_2$는 두 종류의 지수 가중 평균을 조정하는 하이퍼파라미터입니다.
- $\alpha$는 학습률입니다.
- $\varepsilon$은 0으로 나누는 연산을 방지하기 위한 매우 작은 값입니다.

**연습 문제**: Adam 알고리즘을 수행하기 위해, 과거 연산의 정보를 트래킹하는 $v, s$를 초기화해보세요.

**지시 사항** : $v, s$ 변수는 파이썬 딕셔너리로, 모두 0으로 초기화되어야 합니다. 딕셔너리의 키 값은 `grads`와 동일하며, 이는  $l = 1, ..., L$ 로 구성되어 있습니다.

```python
v["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])
v["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])
s["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])
s["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])

```

In [ ]:
# GRADED FUNCTION: initialize_adam

def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...

    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
    ### START CODE HERE ### (approx. 4 lines)
        v["dW" + str(l+1)] = None
        v["db" + str(l+1)] = None
        s["dW" + str(l+1)] = None
        s["db" + str(l+1)] = None
    ### END CODE HERE ###
    
    return v, s

In [ ]:
parameters = initialize_adam_test_case()

v, s = initialize_adam(parameters)
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = \n" + str(v["db2"]))
print("s[\"dW1\"] = \n" + str(s["dW1"]))
print("s[\"db1\"] = \n" + str(s["db1"]))
print("s[\"dW2\"] = \n" + str(s["dW2"]))
print("s[\"db2\"] = \n" + str(s["db2"]))

**모범 답안**:

```
v["dW1"] = 
[[ 0.  0.  0.]
 [ 0.  0.  0.]]
v["db1"] = 
[[ 0.]
 [ 0.]]
v["dW2"] = 
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
v["db2"] = 
[[ 0.]
 [ 0.]
 [ 0.]]
s["dW1"] = 
[[ 0.  0.  0.]
 [ 0.  0.  0.]]
s["db1"] = 
[[ 0.]
 [ 0.]]
s["dW2"] = 
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
s["db2"] = 
[[ 0.]
 [ 0.]
 [ 0.]]
```

**연습 문제**: 이제, Adam 알고리즘을 사용한 파라미터 초기화 로직을 구현해보세요. 위에서 사용했던 업데이트 공식을 그대로 구현하면 됩니다.

$l = 1, ..., L$인 $l$에 대하여,

$$\begin{cases}
v_{W^{[l]}} = \beta_1 v_{W^{[l]}} + (1 - \beta_1) \frac{\partial J }{ \partial W^{[l]} } \\
v^{corrected}_{W^{[l]}} = \frac{v_{W^{[l]}}}{1 - (\beta_1)^t} \\
s_{W^{[l]}} = \beta_2 s_{W^{[l]}} + (1 - \beta_2) (\frac{\partial J }{\partial W^{[l]} })^2 \\
s^{corrected}_{W^{[l]}} = \frac{s_{W^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{W^{[l]}}}{\sqrt{s^{corrected}_{W^{[l]}}}+\varepsilon}
\end{cases}$$

**주의**: for 문에서의 반복 변수 `l`은 0부터 시작하는데 반해 가중치 및 bias 행렬은 $W^{[1]}$ 과 $b^{[1]}$ 로 1부터 시작한다는 것을 주의하세요. for문에서 `l` 을 `l+1` 로 바꿔야 합니다.


In [ ]:
# GRADED FUNCTION: update_parameters_with_adam

def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
        ### START CODE HERE ### (approx. 2 lines)
        v["dW" + str(l+1)] = None
        v["db" + str(l+1)] = None
        ### END CODE HERE ###

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        v_corrected["dW" + str(l+1)] = None
        v_corrected["db" + str(l+1)] = None
        ### END CODE HERE ###

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        ### START CODE HERE ### (approx. 2 lines)
        s["dW" + str(l+1)] = None
        s["db" + str(l+1)] = None
        ### END CODE HERE ###

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        s_corrected["dW" + str(l+1)] = None
        s_corrected["db" + str(l+1)] = None
        ### END CODE HERE ###

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = None
        parameters["b" + str(l+1)] = None
        ### END CODE HERE ###

    return parameters, v, s

In [ ]:
parameters, grads, v, s = update_parameters_with_adam_test_case()
parameters, v, s  = update_parameters_with_adam(parameters, grads, v, s, t = 2)

print("W1 = \n" + str(parameters["W1"]))
print("b1 = \n" + str(parameters["b1"]))
print("W2 = \n" + str(parameters["W2"]))
print("b2 = \n" + str(parameters["b2"]))
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = \n" + str(v["db2"]))
print("s[\"dW1\"] = \n" + str(s["dW1"]))
print("s[\"db1\"] = \n" + str(s["db1"]))
print("s[\"dW2\"] = \n" + str(s["dW2"]))
print("s[\"db2\"] = \n" + str(s["db2"]))

**모범 답안**:

```
W1 = 
[[ 1.63178673 -0.61919778 -0.53561312]
 [-1.08040999  0.85796626 -2.29409733]]
b1 = 
[[ 1.75225313]
 [-0.75376553]]
W2 = 
[[ 0.32648046 -0.25681174  1.46954931]
 [-2.05269934 -0.31497584 -0.37661299]
 [ 1.14121081 -1.09245036 -0.16498684]]
b2 = 
[[-0.88529978]
 [ 0.03477238]
 [ 0.57537385]]
v["dW1"] = 
[[-0.11006192  0.11447237  0.09015907]
 [ 0.05024943  0.09008559 -0.06837279]]
v["db1"] = 
[[-0.01228902]
 [-0.09357694]]
v["dW2"] = 
[[-0.02678881  0.05303555 -0.06916608]
 [-0.03967535 -0.06871727 -0.08452056]
 [-0.06712461 -0.00126646 -0.11173103]]
v["db2"] = 
[[ 0.02344157]
 [ 0.16598022]
 [ 0.07420442]]
s["dW1"] = 
[[ 0.00121136  0.00131039  0.00081287]
 [ 0.0002525   0.00081154  0.00046748]]
s["db1"] = 
[[  1.51020075e-05]
 [  8.75664434e-04]]
s["dW2"] = 
[[  7.17640232e-05   2.81276921e-04   4.78394595e-04]
 [  1.57413361e-04   4.72206320e-04   7.14372576e-04]
 [  4.50571368e-04   1.60392066e-07   1.24838242e-03]]
s["db2"] = 
[[  5.49507194e-05]
 [  2.75494327e-03]
 [  5.50629536e-04]]
```

지금까지 세 가지 작업 최적화 알고리즘 (미니 배치 경사 하강 법, Momentum, Adam)을 구현했습니다. 이러한 각 최적화 함수로 모델을 구현하고 차이점을 관찰 해 보겠습니다.

## 5. 서로 다른 최적화 함수로 모델 테스트하기 ##

아래 "달" 모양 데이터 셋을 사용하여 앞서 구현했던 다양한 최적화 방법을 테스트해보겠습니다. (두 클래스 각각의 데이터가 초승달 모양 처럼 보이기 때문에 데이터 세트 이름이 *moons* 입니다)

In [ ]:
train_X, train_Y = load_dataset()

사전에 구현된 3-layer의 모델에 아래 세 가지의 최적화 함수를 적용해보겠습니다.
- **미니 배치 경사 하강법** : `update_parameters_with_gd()`를 호출합니다.
- **미니 배치 경사 하강법 + 모멘텀** : `initialize_velocity()` 메소드를 호출하고 나서, `update_parameters_with_momentum()` 메소드를 호출합니다.
- **미니 배치 경사 하강법 + Adam 알고리즘**: `initialize_adam()` 메소드 호출 이후 `update_parameters_with_adam()` 호출

In [ ]:
def model(X, Y, layers_dims, optimizer, learning_rate = 0.0007, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):
    """
    3-layer neural network model which can be run in different optimizer modes.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    layers_dims -- python list, containing the size of each layer
    learning_rate -- the learning rate, scalar.
    mini_batch_size -- the size of a mini batch
    beta -- Momentum hyperparameter
    beta1 -- Exponential decay hyperparameter for the past gradients estimates 
    beta2 -- Exponential decay hyperparameter for the past squared gradients estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates
    num_epochs -- number of epochs
    print_cost -- True to print the cost every 1000 epochs

    Returns:
    parameters -- python dictionary containing your updated parameters 
    """

    L = len(layers_dims)             # number of layers in the neural networks
    costs = []                       # to keep track of the cost
    t = 0                            # initializing the counter required for Adam update
    seed = 10                        # For grading purposes, so that your "random" minibatches are the same as ours
    m = X.shape[1]                   # number of training examples
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    # Initialize the optimizer
    if optimizer == "gd":
        pass # no initialization required for gradient descent
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # Optimization loop
    for i in range(num_epochs):
        
        # Define the random minibatches. We increment the seed to reshuffle differently the dataset after each epoch
        seed = seed + 1
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
        cost_total = 0
        
        for minibatch in minibatches:

            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            a3, caches = forward_propagation(minibatch_X, parameters)

            # Compute cost and add to the cost total
            cost_total += compute_cost(a3, minibatch_Y)

            # Backward propagation
            grads = backward_propagation(minibatch_X, minibatch_Y, caches)

            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters_with_gd(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        cost_avg = cost_total / m
        
        # Print the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_avg))
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)
                
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()

    return parameters

이제 3-layer의 모델을 서로 다른 세 개의 최적화 함수로 훈련시긴 모델을 통해, 서로의 차이를 비교해보겠습니다.

### 5-1. 미니 배치 경사 하강법 ###

아래 코드 블록을 실생시켜 미니 배치 경사 하강법으로 훈련시킨 모델이 어떻게 생겼는지 확인해보세요.

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, optimizer = "gd")

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Gradient Descent optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5-2. 미니 배치 경사 하강법 + 모멘텀 ###

아래 코드를 실행시켜 모멘텀을 추가한 모델이 어떤 양상을 보이는지 확인해보세요. 이 학습 데이터는 비교적 단순하기 때문에 모멘텀으로부터 얻을 수 있는 이득이 적지만, 주어진 문제가 어려우면 어려울수록 모멘텀으로 인해 얻을 이익은 커집니다.

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, beta = 0.9, optimizer = "momentum")

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Momentum optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5-3. 미니 배치 경사 하강법 + Adam 최적화 알고리즘 ###

아래 코드를 실행시켜 Adam 최적화 알고리즘을 가미한 모델이 어떻게 생겼는지 확인하세요.

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, optimizer = "adam")

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Adam optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5-4. 요약 ###

<table> 
    <tr>
        <td>
        <b>optimization method</b>
        </td>
        <td>
        <b>accuracy</b>
        </td>
        <td>
        <b>cost shape</b>
        </td>
    </tr>
        <td>
        Gradient descent
        </td>
        <td>
        79.7%
        </td>
        <td>
        oscillations
        </td>
    <tr>
        <td>
        Momentum
        </td>
        <td>
        79.7%
        </td>
        <td>
        oscillations
        </td>
    </tr>
    <tr>
        <td>
        Adam
        </td>
        <td>
        94%
        </td>
        <td>
        smoother
        </td>
    </tr>
</table> 

모멘텀 변수는 유용하지만, 이번 과제에서의 작은 학습률과 단순한 훈련 데이터셋을 고려했을 때 모멘텀 변수의 효과는 거의 무시해도 될 수준입니다. 물론 비용 그래프에서 볼 수 있는 큰 진동은 최적화 알고리즘에서 일부 미니 배치가 다른 미니 배치보다 다루기 어렵다는 것을 보여줍니다.

반면 Adam 알고리즘의 경우, 분명이 일반적인 미니 배치 경사 하강법과 모멘텀에 비해 월등한 결과를 보여줍니다. 만약 우리가 모델을 이 단순한 데이터 셋에 대해 더 많은 횟수로(epochs) 훈련했다면 세 알고리즘 모두 훌륭한 결과를 냈을 것입니다. 하지만 Adam 알고리즘이 보다 더 빨리 최솟값으로 수렴하는 것을 볼 수 있습니다.

**Adam 알고리즘을 사용햇을 때의 이점은,**
- 일반적인 경사 하강법과 모멘텀에 비해 비교적 적은 메모리를 사용합니다.
- 하이퍼파라미터를 조금만 튜닝해도 잘 동작합니다 (학습률 $\alpha$를 제외하고)

**참고 문헌**:

- Adam paper: https://arxiv.org/pdf/1412.6980.pdf